In [444]:
import numpy as np
import os
import glob as gb
import matplotlib.pyplot as plt
from tqdm import tqdm
import cv2
import matplotlib.pyplot as plt

In [445]:
TRAIN_PATH ='../input/cerveau/training/'
IMG_WIDTH = 256
IMG_HEIGHT = 256
IMG_CHANNELS = 1
X_train=[]
Y_train=[]
for folder in tqdm(os.listdir(TRAIN_PATH)):
    files=gb.glob(TRAIN_PATH+'/images/*.png')
    for file in files :
        img =cv2.imread(file,0)
        img = cv2.resize(img, (IMG_HEIGHT, IMG_WIDTH)) 
        X_train.append(list(img)) 
    files=gb.glob(TRAIN_PATH+'/mask/*.png')
    for file in files :
        img =cv2.imread(file,0)
        img = cv2.resize(img, (IMG_HEIGHT, IMG_WIDTH)) 
        Y_train.append(list(img)) 

In [446]:
# a = np.array([2,4,40])
os.listdir(TRAIN_PATH)
len(gb.glob(TRAIN_PATH+'/mask/*.png'))


In [447]:
X_train=np.array(X_train)
X_train = np.reshape(X_train,(X_train.shape[0],X_train.shape[1],X_train.shape[2],1))
Y_train=np.array(Y_train,np.bool)
X_train.shape
# plt.imshow(X_train[1],cmap='gray')


In [448]:
plt.figure(figsize=(20,20))
for n ,i in enumerate(list(np.random.randint(0,len(X_train),16))) :
    plt.subplot(4,4,n+1)
    plt.imshow(X_train[i],cmap='gray')
    plt.axis('off')
    plt.title(i)

In [449]:
plt.figure(figsize=(20,20))
for n ,i in enumerate(list(np.random.randint(0,len(Y_train),16))) :
    plt.subplot(4,4,n+1)
    plt.imshow(Y_train[i],cmap='gray')
    plt.axis('off')
    plt.title(i)

In [450]:
!pip install pydicom

In [451]:
import pydicom as dicom
from pydicom.data import get_testdata_file
#plt.imshow(pixel_array_numpy,cmap=plt.cm.bone)

In [452]:
TRAIN_PATH ='../input/cerveau/test/'
X_test=[]
for folder in tqdm(os.listdir(TRAIN_PATH)):
    files=gb.glob(TRAIN_PATH+'images/*.png')
    for file in files :
        img =cv2.imread(file,0)
        img = cv2.resize(img, (IMG_HEIGHT, IMG_WIDTH)) 
        X_test.append(list(img))
        
#     for file in files :
#         img =cv2.imread(file,0)
#         img = cv2.resize(img, (IMG_HEIGHT, IMG_WIDTH)) 
#         X_train.append(list(img))

In [453]:
X_test=np.array(X_test)
X_test = np.reshape(X_test,(X_test.shape[0],X_test.shape[1],X_test.shape[2],1))
X_test.shape

In [454]:
plt.figure(figsize=(20,20))
for n ,i in enumerate(list(np.random.randint(0,len(X_test),16))) :
    plt.subplot(4,4,n+1)
    plt.imshow(X_test[i],cmap='gray')
    plt.axis('off')
    plt.title(i)

In [455]:
from tensorflow.keras.models import Model 
from tensorflow.keras.layers import Conv2D , MaxPooling2D ,concatenate ,Input ,Dropout ,Conv2DTranspose
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K

In [456]:
def dice_score(ytrue, ypred):
    smoothing_factor=0.1
    ytrue_f = K.flatten(ytrue)
    ypred_f = K.flatten(ypred)
    intersection = K.sum(ytrue*ypred)
    ytrue_area = K.sum(ytrue)
    ypred_area = K.sum(ypred)
    combined_area = ytrue_area + ypred_area
    dice = 2*((intersection+smoothing_factor)/(combined_area+smoothing_factor))
    return dice


In [457]:
def iou(ytrue, ypred):
    smoothing_factor=0.1
    #y_true_f=K.flatten(y_true)
    #y_pred_f=K.flatten(y_pred)
    intersection = K.sum(ytrue*ypred)
    combined_area = K.sum(ytrue+ypred)
    union_area = combined_area - intersection
    iou = (intersection+smoothing_factor)/(union_area+smoothing_factor)
    return iou

In [458]:
def iou_loss(ytrue, ypred):
    return -iou(ytrue, ypred)

In [459]:
def unet(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS):
#Build the model
    inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
    #s = Lambda(lambda x: x / 255)(inputs)   #No need for this if we normalize our inputs beforehand
    s = inputs

    #Contraction path
    
    
    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(s)
    c2 = Dropout(0.1)(c2)
    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)
     
    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
    c3 = Dropout(0.2)(c3)
    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)
     
    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
    c4 = Dropout(0.2)(c4)
    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
    p4 = MaxPooling2D(pool_size=(2, 2))(c4)
     
    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
    c5 = Dropout(0.3)(c5)
    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)
    
    #Expansive path 
    u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
    c6 = Dropout(0.2)(c6)
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)
     
    u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
    c7 = Dropout(0.2)(c7)
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)
     
    u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
    c8 = Dropout(0.1)(c8)
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)
     
    
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c8)
     
    model = Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy',iou,dice_score])

    model.summary()
    
    return model

In [460]:
model=unet(IMG_HEIGHT,IMG_WIDTH,IMG_CHANNELS)

In [461]:
from tensorflow.keras.callbacks import EarlyStopping ,ReduceLROnPlateau ,ModelCheckpoint

# EarlyStop=EarlyStopping(patience=10,restore_best_weights=True)
Reduce_LR=ReduceLROnPlateau(monitor='val_accuracy',verbose=2,factor=0.5,min_lr=0.00001)
model_check=ModelCheckpoint('model.hdf5',monitor='val_loss',verbose=1,save_best_only=True)
callback=[Reduce_LR,model_check]

# X_train


In [462]:
# !pip install keras
# pour générer des images altere
#from tensorflow import keras 
# from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator



# Image Data Augmentation
train_generator = ImageDataGenerator(
    rotation_range=2, horizontal_flip=True, zoom_range=0.1
)

val_generator = ImageDataGenerator(
    rotation_range=2, horizontal_flip=True, zoom_range=0.1
)

test_generator = ImageDataGenerator(
    rotation_range=2, horizontal_flip=True, zoom_range=0.1
)

# Fitting the augmentation defined above to the data
train_generator.fit(X_train)
# val_generator.fit(x_val)
test_generator.fit(X_test)
# X_train

In [463]:
# history=model.fit(X_train,Y_train,validation_split=0.1,batch_size=32,steps_per_epoch=len(X_train)//32,epochs=500,
#                  callbacks=callback,verbose=1)

history=model.fit(X_train,Y_train,validation_split=0.1,steps_per_epoch=len(X_train)//16,epochs=100,
                  callbacks=callback,verbose=1)

In [464]:
y_pred=model.predict(X_train)

In [465]:
y_pred_p=(y_pred > 0.5).astype(np.uint8)

In [466]:
plt.imshow(y_pred_p[0],cmap='gray')

In [467]:
iou = history.history['iou']
val_iou = history.history['val_loss']
epochs = range(1,len(loss)+1)
plt.plot(epochs,loss,'y',label='Training loss')
plt.plot(epochs, val_loss,'r',label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
plt.plot(epochs,acc,'y',label='Training acc')
plt.plot(epochs, val_acc,'r',label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()

In [468]:
iou = history.history['iou']
val_iou = history.history['val_iou']
epochs = range(1,len(iou)+1)
plt.plot(epochs,iou,'y',label='Training iou')
plt.plot(epochs, val_iou,'r',label='Validation iou')
plt.title('Training and validation iou')
plt.xlabel('Epochs')
plt.ylabel('iou')
plt.legend()
plt.show()

dice_score = history.history['dice_score']
val_dice_score = history.history['val_dice_score']
plt.plot(epochs,dice_score,'y',label='Training dice_score')
plt.plot(epochs, val_dice_score,'r',label='Validation dice_score')
plt.title('Training and validation dice_score')
plt.xlabel('Epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()